# Gettin Evaluation Framework for Learning Analytics (EFLA)

* **author** = Diego Sapunar-Opazo
* **copyright** = Copyright 2019, Thesis M.Sc. Diego Sapunar - Pontificia Universidad Católica de Chile
* **credits** = Diego Sapunar-Opazo, Ronald Perez, Mar Perez-Sanagustin, Jorge Maldonado-Mahauad
* **maintainer** = Diego Sapunar-Opazo
* **email** = dasapunar@uc.cl
* **status** = Dev

Source: http://www.laceproject.eu/evaluation-framework-for-la/

The Evaluation Framework for Learning Analytics (EFLA) addresses the current lack of evaluation instruments by offering a standardised way to evaluate learning analytics tools and to measure and compare the impact of learning analytics on educational practices of learners and teachers. The EFLA makes use of the subjective assessments about learning analytics tools by their users in order to obtain a general indication of the overall quality of a tool in a quick and simple, yet thoroughly developed, validated and reliable way.

The EFLA consists of the three dimensions Data, Awareness & Reflection, and Impact with a total of eight items. There is one version for learners and one version for teachers. All items are to be rated on a scale from 1 for ‘strongly disagree’ to 10 for ‘strongly agree’. The EFLA score can be any number between 0 and 100. In order to calculate a learning analytics tool’s EFLA score, the following steps should be taken per stakeholder group:

(1) calculate the average value for each item based on the answers given for that item,

(2) calculate the average value for each dimension based on the average of its items,

(3) calculate the dimensional scores by rounding the result of ((x-1)/9)*100 where x is the average value of a dimension (in order to get a number between 0 and 100), and

(4) calculate the overall EFLA score by taking the average of the three dimensional scores.

This script gets a survey done un Survey Monkey to the students and implement the framework metioned above, creating a csv file with:

(1) **num_alumno**, which corresponds to the internal face-to-face students' id

(2) **data_value**, which corresponds to the value of the Data dimension

(3) **data_score**, which corresponds to the EFLA score of the Data dimension

(4) **a&r_value**, which corresponds to the value of the Awareness & Reflection(a&r) dimension

(5) **a&r_score**, which corresponds to the EFLA score of the Awareness & Reflection(a&r) dimension

(6) **impact_value**, which corresponds to the value of the Impact dimension

(7) **impact_score**, which corresponds to the EFLA score of the Impact dimension

(8) **EFLA_score**, which corresponds to the EFLA score for that student

(9) **recommend**, which is a boolean if would recommend NMP

## Part 0: Import Packages

In [2]:
# data analysis and wrangling
import pandas as pd
import numpy as np

## Part 1: Getting the Data

In [8]:
def read_data(path, header=False):
    '''
    Read a .csv file and convert it in a Pandas DataFrame.
    
    Input:
    path - String: path where the .csv is located.
    
    Output:
    Pandas DataFrame: .csv in the Pandas DataFrame format.
    '''
    if header:
        return pd.read_csv(path, header=header)        
    else:
        return pd.read_csv(path)

## Part 2: Data Preprocessing

In [67]:
def preprocc_data(df, slices=False, columns_to_rename=False, categories=False, values_to_replace=False):
    '''
    From a dataframe on the fly, (1) get the necessary columns; (2) rename columns; and (3) clean data.
    
    Input: 
    df - Pandas DataFrame: dataframe to be cleaned.
    columns_to_rename - Dict: Columns to rename, Key: original name, Value: new name.
    categories - List of Strings: List of the names of the columns to be category type. If you renamed some columns, should be the new names.
    values_to_replace - Dict- Values to replace, Key: original value, Value: new value.
    
    Output:
    df - Pandas DataFrame: the dataframe already cleaned.
    '''
    
    df_cleaned = df.copy()
    
    # slicing the columns, getting only the one that I need (num_alumno and seccion)
    if slices:
        df_cleaned = df_cleaned.iloc[:,10:-2]
    
    del df  # clean memory
    
    # rename columns
    if columns_to_rename:
        for index, col in enumerate(df_cleaned.columns):
            if index == 0: pass
            elif index == 9: columns_to_rename[col] = 'recommend'
            else: columns_to_rename[col] = 'Q' + str(index)
        
        df_cleaned.rename(columns_to_rename, 
                          inplace=True, 
                          axis=1)
    
    if categories:
        for cat in categories:
            # creating categories
            df_cleaned[cat] = df_cleaned[cat].astype('category')
            
    if values_to_replace:
        df_cleaned.replace(values_to_replace, inplace=True)
    
    return df_cleaned

In [68]:
def EFLA(df_original):
    '''
    Apply EFLA to the cleaned df.
    
    Input:
    df_original - Pandas DataFrame: dataframe to calculate de EFLA.
    
    Output:
    df - Pandas DataFrame: final dataframe.
    '''
    df = df_original.copy()
    
    df['data_value'] = (df['Q1'] + df['Q1'])/2
    df['a&r_value'] = (df['Q3'] + df['Q4'] + df['Q5'] + df['Q6'])/4
    df['impact_value'] = (df['Q7'] + df['Q8'])/2

    df['data_score'] = ((df['data_value'] - 1)/9) * 100
    df['a&r_score'] = ((df['a&r_value'] - 1)/9) * 100
    df['impact_score'] = ((df['impact_value'] - 1)/9) * 100

    df['EFLA_score'] = (df['data_score'] + df['a&r_score'] + df['impact_score'])/3
    
    return df

In [69]:
def merging(df1, df2, variable1, variable2):
    '''
    Merge df1 and df2 over the variable.
    
    Input:
    df1 - Pandas DataFrame
    df2 - Pandas DataFrame
    variable - String: name of the column to use as pivot.
    
    Output:
    Pandas DataFrame
    '''

    df1.dropna(inplace=True)
    df2.dropna(inplace=True)
    
    # getting same types
    df1[variable1] = df1[variable1].astype('str')
    df2[variable2] = df2[variable2].astype('str')
    
    return pd.merge(left=df1, right=df2, left_on=variable1, right_on=variable2)
    

## Part 3: Export Data

In [70]:
def export_data(df, path, columns_to_drop=False):
    '''
    Export df in .csv file to the path.
    
    Input:
    df - Pandas DataFrame: dataframe to be exported.
    path - String: path where the .csv will be exported.
    '''
    if columns_to_drop:
        df.drop(columns_to_drop, axis=1, inplace=True)
        
    df.to_csv(path, index=False)

# Part 4: Main

In [78]:
# getting data
_EFLA_path = '../data/original_data/surveys/EFLA_survey.csv'
df_EFLA = read_data(_EFLA_path, header=1)

_students_emails_path = '../data/final_model/students_email.csv'
df_students_emails = read_data(_students_emails_path)

# preprocc
_columns_to_rename = {'Open-Ended Response': 'student_email'}
_values_to_replace = {'1 (Totalmente en desacuerdo)': 1,
                    '2': 2,
                    '3': 3,
                    '4': 4,
                    '5': 5,
                    '6': 6,
                    '7': 7,
                    '8': 8,
                    '9': 9,
                    '10\xa0(Totalmente de acuerdo)': 10}

df_EFLA = preprocc_data(df_EFLA, 
                        slices=True,
                        columns_to_rename=_columns_to_rename, 
                        categories=['recommend'], 
                        values_to_replace=_values_to_replace)

# merging
df_EFLA = merging(df_EFLA, df_students_emails, 
                  'student_email', 'student_email')

# EFLA
df_EFLA = EFLA(df_EFLA)

# export data
columns_to_drop = ['Q'+str(i) for i in range(1,9)] + ['student_email']
_EFLA_export_path = '../data/final_model/NMP_EFLA.csv'
export_data(df_EFLA, _EFLA_export_path, columns_to_drop)

In [82]:
df_EFLA.head()

,recommend,num_alumno,data_value,a&r_value,impact_value,data_score,a&r_score,impact_score,EFLA_score
0,NO,16622480,3.0,2.00,1.0,22.222222,11.111111,0.000000,11.111111
1,NO,13636596,6.0,4.75,3.5,55.555556,41.666667,27.777778,41.666667
2,NO,1663716J,8.0,4.75,3.0,77.777778,41.666667,22.222222,47.222222
3,NO,15638650,7.0,2.00,1.0,66.666667,11.111111,0.000000,25.925926
4,SI,16623029,6.0,6.00,5.5,55.555556,55.555556,50.000000,53.703704
